In [1]:
import numpy as np
import pandas as pd
import keras
import pickle

Using TensorFlow backend.


In [ ]:
!pip install tensorflow
!pip install keras

In [2]:
import Consultas_entrenar as ent
import obtener_contenido_archivo as oca

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=707090095875-bgbl1m86fr5r0inqg1d01obesaj4109e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
# Recrea exactamente el mismo modelo desde el archivo

new_model = keras.models.load_model('modelo_pred_violencia.h5')

In [ ]:

cv_loaded = pickle.load(open('countVectorizer_spacy', "rb"))

In [3]:
def predecir_violencia(url):
    new_model = keras.models.load_model('modelo_pred_violencia.h5')
    cv_loaded = pickle.load(open('countVectorizer_spacy', "rb"))
    #obtener contenido del archivo
    declaracion= oca.descargar_de_drive([url])
    #Normaliza el texto
    declaracion=pd.DataFrame(declaracion)
    corpus=ent.armar_bag_of_words(declaracion)
    #Aplica CountVectorizer al corpus normalizado
    obse_vec=cv_loaded.transform(corpus).toarray()
    #predecir
    new_predictions = new_model.predict(obse_vec)
    return new_predictions

In [23]:
pred=predecir_violencia('https://drive.google.com/open?id=0B9wNhp3GjjazRnBoRXM0R1BFNlE')

0B9wNhp3GjjazRnBoRXM0R1BFNlE 3165 L. A. SA s. infr. art. 2 ley 26735.pdf application/pdf


In [31]:
resultado=pd.DataFrame(pred>0.5).transpose()
resultado["probabilidad"]=pd.DataFrame(pred.transpose())

In [26]:
del resultado

In [42]:
from unicodedata import normalize

In [99]:
def limpiar_texto(dataset):
    import html
    import re
    review = str(html.unescape(dataset))
    review = cleanhtml(review)
    review = normalize( 'NFC', review)
    review = re.sub('[^a-zA-Zá-ú0-9."",?!:]', ' ', review)
    review = review.split()
    review = [word for word in review if len(word) > 1 or word in set(['a', 'e', 'y', 'o', 'u'])]
    review = ' '.join(review)
    return review


In [95]:
def limpiar_texto2(dataset):
    import html
    import re
    review = str(html.unescape(dataset))
    review = cleanhtml(review)
    
    review = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", review), 0, re.I)
    review = review.split()
    review = [word for word in review if len(word) > 1 or word in set(['a', 'e', 'y', 'o', 'u'])]
    review = ' '.join(review)
    return review


In [96]:

def cleanhtml(raw_html):
    import re
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [104]:
texto='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\']!;i¡<r t\ gt°\'. ": [�. \'f\' imj,v·jj;; ,uenos aires, 14 de septiembre de 2014.\n\ \n\n� d 7;�,/j.\' �/ o/ \n\n\.,: \�\n\'(¡·0"\' j,:\' \n\n\' autos y vistos. \n\\�� /// .. :.- / \n\n\:?;:¡4;,,:\'., ,, \n/ para resolver respecto de la solicitud fiscal en la causa nº \n\n12749/16, del registro de la secretaria general de la camara de apelaciones \n\ndel fuero, en tramite ante este juzgado en lo penal, contravencional y de \nfaltas nº 10, caratulada "a v , s l s/ inf. art. 1, \n\nln 13.944, incumplimiento de los deberes de asistencia familiar p/l \n\n2303". \n\nresulta: \n\n;\' \ni \n\n/\n\nla presente causa tiene por objeto, el hecho que fuera precisado por \n\nla señora fiscal, en el decreto de determinacion de los hechos confeccionado \ncon fecha 11 de julio de 2016 (fs. 15). \n\nen dicha pieza procesal, y a merito de lo dispuesto por el art. 92 del \n\ncodigo de rito, la acusadora publica preciso que la presente investigacion, \n\ntendria por objeto determinar la responsabilidad de s l a \n\nv , por cuanto se habria sustraido de prestar dolosamente los medios \n\nindispensables para la subsistencia de su hija r a a s , \nnacida el 18 de enero de 20, domiciliada junto a su madre c m \n\nq_� s en la finca sita en la calle carlos calvo 3*, habitacion *, de esta\n.,j ciudad, ello desde enero 2016 hasta al menos la actualidad?, al no realizar \n\nc.. a,s \n\'15 i aportes para colaborar con su manutencion, ya sea dinerario o en concepto de \n·-::, \n\n-;;: i alimentos, vestimenta, educacion y/o esparcimiento.\n�b \n� ,))? la'


In [105]:
texto.replace('\n', ' ')

'                                         \']!;i¡<r t\\ gt°\'. ": [�. \'f\' imj,v·jj;; ,uenos aires, 14 de septiembre de 2014. \\   � d 7;�,/j.\' �/ o/   \\.,: \\� \'(¡·0"\' j,:\'   \' autos y vistos.  \\�� /// .. :.- /   \\:?;:¡4;,,:\'., ,,  / para resolver respecto de la solicitud fiscal en la causa nº   12749/16, del registro de la secretaria general de la camara de apelaciones   del fuero, en tramite ante este juzgado en lo penal, contravencional y de  faltas nº 10, caratulada "a v , s l s/ inf. art. 1,   ln 13.944, incumplimiento de los deberes de asistencia familiar p/l   2303".   resulta:   ;\'  i   /  la presente causa tiene por objeto, el hecho que fuera precisado por   la señora fiscal, en el decreto de determinacion de los hechos confeccionado  con fecha 11 de julio de 2016 (fs. 15).   en dicha pieza procesal, y a merito de lo dispuesto por el art. 92 del   codigo de rito, la acusadora publica preciso que la presente investigacion,   tendria por objeto determinar la respons